In [1]:
# Importing necessary Libraries
import pandas as pd
import yfinance as yf
import requests
### These are the libraries we need for writing the data to s3
### https://khandelwal-shekhar.medium.com/read-and-write-to-from-s3-using-python-boto3-and-pandas-s3fs-144341e23aa1
import io   
from io import StringIO
import yaml
import boto3
import os

In [2]:
# config_file path
aws_api_key_path = r"..\\config\\cred_config.yaml"
config_file_path = r"..\\config\\config.yaml"

In [3]:
with open(aws_api_key_path,'r') as aws_api_file:
    aws_api_config = yaml.safe_load(aws_api_file)
    

In [4]:
with open(config_file_path,'r') as config_file:
    config = yaml.safe_load(config_file)

In [5]:
# Argumant for Daily Load and One Time load
oneTime = True


In [6]:
#Configuring the AWS keys
AWS_S3_BUCKET = aws_api_config.get('AWS_S3_BUCKET')
AWS_ACCESS_KEY_ID = aws_api_config.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY  = aws_api_config.get('AWS_SECRET_ACCESS_KEY')
REGION_NAME = aws_api_config.get('REGION_NAME')


In [7]:
#Common Configs
index_output_data_file_name = config.get('index_path_50')[1]
index_output_detailed_file_name = config.get('index_path_50')[2]
index_path = config.get('index_path_50')[0]
excluded_keys_info = config.get('excluded_keys_info') #['companyOfficers']

In [8]:
stocks = pd.read_csv(
    f"s3://{AWS_S3_BUCKET}/{index_output_data_file_name}",
    storage_options = {
        "key" : AWS_ACCESS_KEY_ID,
        "secret" : AWS_SECRET_ACCESS_KEY,
    }
)

In [54]:
script= [x+".NS" for x in stocks["Symbol"]]

In [ ]:
Daily_Data_from_all_stocks = {x:yf.download([x], period='max',interval='1d',group_by='ticker') for x in script}

In [57]:
Weekly_Data_from_all_stocks = {x:yf.download([x], period='max',interval='1wk',group_by='ticker') for x in script}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [58]:
for script in Daily_Data_from_all_stocks:
    Daily_Data_from_all_stocks[script].stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=0).to_csv(
        f"s3://{AWS_S3_BUCKET}/daily_fl/{script}.csv",
        index=False,
        storage_options = {
            "key": AWS_ACCESS_KEY_ID,
            "secret" : AWS_SECRET_ACCESS_KEY,
        }
        )


In [59]:
for script in Weekly_Data_from_all_stocks:
    Weekly_Data_from_all_stocks[script].stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=0).to_csv(
        f"s3://{AWS_S3_BUCKET}/weekly_fl/{script}.csv",
        index=False,
        storage_options = {
            "key": AWS_ACCESS_KEY_ID,
            "secret" : AWS_SECRET_ACCESS_KEY,
        }
        )

In [69]:
 Daily_Data_from_all_stocks['ADANIENT.NS'].stack(level=0,future_stack=True).index[-1]

(Timestamp('2024-11-05 00:00:00+0000', tz='UTC'), 'ADANIENT.NS')